In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import pingouin as pg
import os
import matplotlib.pyplot as plt
import json

# Data cleaning

In [27]:


foldernames = [
    './GPT3/cushman/items-07_07_2022__19:58:42',
    './GPT3/cushman/items-003_2023',
    './HUMANS/',
]
filenames = ['/results.csv', '/results.csv', '/cushman.csv']
labels = ['gpt-002', 'gpt-003', 'human']
dfs = []
dfs_with_unclear = []

dataset = json.load(open('data/cushman/items.json'))
dataset = pd.DataFrame(dataset)
for label, foldername, fname in zip(labels, foldernames, filenames):
    df = pd.read_csv(open(foldername+fname, 'r'))
    df['exp'] = label
    if 'answer' in df.columns:
        df['a'] = df['answer']
    df['a'] = df['a'].str.lower()
    df = df[df['q_id']==0]
    df = df[~df['item_id'].isin([1, 2])]
    for i in df['item_id'].unique():
        if i not in (1, 2):

            df.loc[df['item_id']==i, 'title'] = \
                str(dataset[dataset['id']==i]['title'].unique()[0])[2:]
            df.loc[df['item_id']==i, 'cond'] = \
                str(dataset[dataset['id']==i]['title'].unique()[0])[:2]

    df['principle'] = ''
    df['principle'] = df['principle'].astype('object')
    df.loc[df['cond'].str.contains('ik|ci|fk|cf'), 'principle'] += ' action '
    df.loc[df['cond'].str.contains('l'), 'principle'] += ' inaction '
    df.loc[df['cond'].str.contains('f'), 'principle'] += ' foreseen '
    df.loc[df['cond'].str.contains('i'), 'principle'] += ' intended '
    df.loc[df['cond'].str.contains('c'), 'principle'] += ' contact '
    df.loc[~df['cond'].str.contains('c'), 'principle'] += ' no_contact '
    df['yes'] = df['a'].str.contains(pat=r'\b(yes)\b', regex=True)
    df['no'] = df['a'].str.contains(pat=r'\b(no)\b', regex=True)
    df['unclear'] = df['yes'] == df['no']
    df['val'] = None
    df.loc[df['yes']==1, 'val'] = 'yes'
    df.loc[df['no']==1, 'val'] = 'no'
    df.loc[df['unclear']==1, 'val'] = 'unclear'
    dfs_with_unclear.append(df.copy())
    df = df[df['unclear']!=1] 
    dfs.append(df.copy())
    
df = pd.concat(dfs)
df_with_unclear = pd.concat(dfs_with_unclear)

def mean_by_exp_and_sub(df):
    # set cond label and merge 
    exp = df['exp'].unique()
    dfs = []
    for e in exp:
        d = df[df['exp']==e].groupby(['title', 'principle' ], as_index=False).mean()
        d['exp'] = e
        dfs.append(d.copy())

    df2 = pd.concat(dfs)
    return df2

df2 = mean_by_exp_and_sub(df)
df2_with_unclear = mean_by_exp_and_sub(df_with_unclear)

    
df[df['val']=='unclear'].to_excel(foldername[0] + '/unclear.xlsx')

print('unclear proportion: ', np.mean(df_with_unclear[df_with_unclear.exp=='human']['unclear']==1))

df = df[df['exp'] == 'human']

df_with_unclear.prolific_id.value_counts()

unclear proportion:  0.08915145005370569


/tmp/ipykernel_43691/2428368908.py:37: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  df['yes'] = df['a'].str.contains(pat=r'\b(yes)\b', regex=True)
/tmp/ipykernel_43691/2428368908.py:38: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  df['no'] = df['a'].str.contains(pat=r'\b(no)\b', regex=True)


60c4691787a92ea2736450d4    30
62348a574fc351980c10604d    30
62ee4337445b044245a71185    30
576745749dcd970001dbdd0b    30
5f047ad3b4eb766c94bb20a7    30
5f7e292bc98b530dc2c16c4d    30
5d2e06eb64970300169c8fdf    30
6131f6711c0d96f6f1fb04af    30
60e962df6e5dab0795cae4c6    30
626d611a6d646b36a7721304    30
60c9395dd973ea5ebe0ea6ad    30
60fff225fb2da6129caf029a    30
5e482352020bde2be9b4ded1    30
5f5bb19873fa1f341a4eaa3b    30
5e249bc4292336713bc67cc3    30
62cd7356ccb2b76b1a9d7083    30
5c7093155ba5060001ce5f1a    30
5b52fb6cb3c9400001f23bb5    30
60dd669a2f3a194e6ca03757    30
5c366fc38821900001b38b67    30
5e25d8927ea9c201db8a2fd9    30
60fc7a2f25df85c07054ce04    30
627a83761978ca1469e6e750    30
62ed82df460d7a1d965535b9    30
6294990ec8b96a6df740a8b8    30
60cf508d719c605fdbb3387e    30
615dfbfde9859801c5fc4b87    30
5f2b348c2eeba5556920b2d5    30
5ea07fc202b8bd05bb59bc38    30
5b661bc3259daf00019369f2    29
60d98308ecc4c4e732c86a71    29
5e8cb846e94a4406307d6da2     2
notfound

# Fig 1

In [ ]:
#d = []
#pairs = [('action', 'inaction'), ('intended', 'foreseen'), ('contact', 'no_contact')]
#labels = ('action', 'intention', 'contact')
#titles = [
#    ('Boxcar', 'Pond', 'Ship', 'Car', 'Boat', 'Switch'),
#    ('Speed', 'Burn', 'Boxcar', 'Switch', 'Chem', 'Shark'),
#    ('Speed', 'Boxcar', 'Aqua', 'Rubble')
#]
#for p,l,t1 in zip(pairs, labels, titles):
#    for t in t1:
#        d.append({
#            'principle': p[0],
#            'item': t,
#            'yes': df2.loc[(df2['title'] == t) * (df2['principle'].str.contains(' ' + p[0] + ' ')), 'yes'].mean()
#        })
#        d.append({
#               'principle': p[1],
#               'item': t,
#               'yes': df2.loc[(df2['title'] == t) * (df2['principle'].str.contains(' ' + p[1] + ' ')), 'yes'].mean()
#        })
#
#
#new_df = pd.DataFrame(d)
#new_df
#
#d = []
#
#for p,l,t1 in zip(pairs, labels, titles):
#    for t in t1:
#        d.append(
#            {
#                'principle': p[0],    
#                'item': t,
#                'replicates': df2.loc[(df2['title'] == t) * (df2['principle'].str.contains(' ' + p[1] + ' ')), 'yes'].mean() >\
#                            df2.loc[(df2['title'] == t) * (df2['principle'].str.contains(' ' + p[0] + ' ')), 'yes'].mean()
#
#            }
#        )
#df_table = pd.DataFrame(d)
#df_table.loc[df_table['principle'] == 'intended', 'principle'] = 'intention'
#print(df_table)
#
#
df2 = df
dv = 'yes'
d = []
pairs = [('action', 'inaction'), ('intended', 'foreseen'), ('contact', 'no_contact')]
labels = ('action', 'intention', 'contact')
titles = [
    ('Boxcar', 'Pond', 'Ship', 'Car', 'Boat', 'Switch'),
    ('Speed', 'Burn', 'Boxcar', 'Switch', 'Chem', 'Shark'),
    ('Speed', 'Boxcar', 'Aqua', 'Rubble')
]
for p,l,t1 in zip(pairs, labels, titles):
    for e in ['gpt-002', 'gpt-003', 'human']:
        df2 = df[df['exp']==e]

        if 'gpt' in e:
            key = 'iter'
            sub = df['iter'].unique()
        else:
            key = 'prolific_id'
            sub = df['prolific_id'].unique()


        for t in t1:
            for i in sub:
                if i in ['nan', np.NaN, 'notfound']:
                    continue


                d.append({
                    'principle': p[0],
                    'item': t,
                    'val': df2.loc[(df2[key] == i) * (df2['title'] == t) * (df2['principle'].str.contains(' ' + p[0] + ' ')), dv].mean(),
                    'meta_principle': l,
                    key: i,
                    'exp': e,
                })
                d.append({
                       'principle': p[1],
                       'item': t,
                       'val': df2.loc[(df2[key]==i) * (df2['title'] == t) * (df2['principle'].str.contains(' ' + p[1] + ' ')), dv].mean(),
                       'meta_principle': l,
                       key: i,
                        'exp': e,
                })


new_df = pd.DataFrame(d)

d = []

for p,l,t1 in zip(pairs, labels, titles):
    for e in ['gpt-002', 'gpt-003', 'human']:
        df2 = df[df['exp']==e]


        for t in t1:
            d.append(
                {
                    'principle': p[0],    
                    'item': t,
                    'replicates': df2.loc[(df2['title'] == t) * (df2['principle'].str.contains(' ' + p[1] + ' ')), dv].mean() >\
                                df2.loc[(df2['title'] == t) * (df2['principle'].str.contains(' ' + p[0] + ' ')), dv].mean(),
                                'exp': e,

                }
            )
df_table = pd.DataFrame(d)
df_table.loc[df_table['principle'] == 'intended', 'principle'] = 'intention'
print(df_table)
print(new_df)

# Fig 1

In [ ]:
sns.set(rc={'figure.figsize':(19, 12)})

plt.subplot(1, 3, 1)
df3 = new_df[new_df.exp=='human'].groupby(['principle', 'prolific_id', 'meta_principle'], as_index=False).mean()
df3['principle_variation'] = 'yes'
df3.loc[df3['principle'].isin(['no_contact', 'foreseen', 'inaction']), 'principle_variation'] = 'no'
sns.stripplot(x='meta_principle', y='val', hue='principle_variation', data=df3, size=10, linewidth=1.2, zorder=2, edgecolor='white', dodge=True)
sns.barplot(x='meta_principle', y='val', hue='principle_variation', ci=68,data=df3, alpha=.5, zorder=1, dodge=True)
# raincloud(x='exp', y='yes', markersize=9, df=df2)
x_lim = plt.gca().get_xlim()
plt.plot(x_lim, [.5, .5], ls='--', color='gray', zorder=0)
# plt.ylim(0.5,1.5)
plt.xlabel('Principle pairs')
plt.ylabel('Endorsement')
plt.title('Human')


plt.subplot(1, 3, 2)
df3 = new_df[new_df.exp=='gpt-002'].groupby(['principle', 'iter', 'meta_principle'], as_index=False).mean()
df3['principle_variation'] = 'yes'
df3.loc[df3['principle'].isin(['no_contact', 'foreseen', 'inaction']), 'principle_variation'] = 'no'
sns.stripplot(x='meta_principle', y='val', hue='principle_variation', data=df3, size=10, linewidth=1.2, zorder=2, edgecolor='white', dodge=True)
sns.barplot(x='meta_principle', y='val', hue='principle_variation', data=df3, alpha=.5, zorder=1, ci=68, dodge=True)
# raincloud(x='exp', y='yes', markersize=9, df=df2)
x_lim = plt.gca().get_xlim()
plt.plot(x_lim, [.5, .5], ls='--', color='gray', zorder=0)
# plt.ylim(0.5,1.5)
plt.xlabel('Principle pairs')
plt.ylabel('Endorsement')
plt.title('davinci-002')

plt.subplot(1, 3, 3)
df3 = new_df[new_df.exp=='gpt-003'].groupby(['principle', 'iter', 'meta_principle'], as_index=False).mean()
df3['principle_variation'] = 'yes'
df3.loc[df3['principle'].isin(['no_contact', 'foreseen', 'inaction']), 'principle_variation'] = 'no'
sns.stripplot(x='meta_principle', y='val', hue='principle_variation', data=df3, size=10, linewidth=1.2, zorder=2, edgecolor='white', dodge=True)
sns.barplot(x='meta_principle', y='val', hue='principle_variation', data=df3, alpha=.5, zorder=1, ci=68, dodge=True)
# raincloud(x='exp', y='yes', markersize=9, df=df2)
x_lim = plt.gca().get_xlim()
plt.plot(x_lim, [.5, .5], ls='--', color='gray', zorder=0)
# plt.ylim(0.5,1.5)
plt.xlabel('Principle pairs')
plt.ylabel('Endorsement')
plt.title('davinci-003')




df3 = new_df[new_df.exp=='human'].groupby(['principle', 'prolific_id', 'meta_principle'], as_index=False).mean()
df3['principle_variation'] = 'yes'
df3.loc[df3['principle'].isin(['no_contact', 'foreseen', 'inaction']), 'principle_variation'] = 'no'

pg.ttest(x=df3[df3.principle_variation=='yes'][df3.meta_principle=='contact']['val'], y=df3[df3.principle_variation=='no'][df3.meta_principle=='contact']['val'], paired=True)


In [ ]:
sns.set(rc={'figure.figsize':(14, 8)})

order = ['action', 'inaction', 'intended', 'foreseen', 'contact', 'no_contact']
df3 = new_df.groupby(['principle', 'item'], as_index=False).mean()

sns.stripplot(x='principle', y='val', hue='item', data=df3,
    size=10, linewidth=1.2, zorder=2, edgecolor='white', dodge=True, order=order)

g = sns.barplot(x='principle', y='val', hue='item', data=df3,
    alpha=.5, zorder=1, ci=68, dodge=True, order=order,)

# s.legend_.show()
# raincloud(x='exp', y='yes', markersize=9, df=df2)
x_lim = plt.gca().get_xlim()
plt.plot(x_lim, [.5, .5], ls='--', color='gray', zorder=0)
# plt.ylim(0.5,7.5)
plt.xlabel('Principle')
plt.ylabel('Endorsement')

h, l = g.get_legend_handles_labels()
plt.legend(h[:12], l[:12], loc='lower right')


In [ ]:

def f(s, v, props): return np.where(s.isin([v]), props, '')
s = df_table.style
df_table1 = df_table[df_table['exp'] == 'human']
df_table1.loc[df_table1['replicates']==True, 'replicates'] = '✅'
df_table1.loc[df_table1['replicates']==False, 'replicates'] = '❌'

df_table1
# s.apply(f, v=True, props="color: lightgreen;", axis=0)\
# .apply(f, v=False, props="color: white; background-color:pink", axis=0)
# df_table.style.apply(f, v=False, props="color:lightred;", axis=0)

In [ ]:
ddd = df[(df.exp=='human') & (df.title=='Car')& (df.principle.str.contains('action'))]
